In [1]:
# -*- coding: utf8 -*-
from os.path import expanduser
import codecs,yaml
import itertools as it
import ParFuMor as PFM

In [2]:
home = expanduser("~")
annee="17"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
nomTraductions=repertoire+"Phrases-B2.csv"
nomLexique=repertoire+"%s-K5/"%annee+"stems.yaml"

In [3]:
with codecs.open(nomTraductions,"r",encoding="utf8") as input:
    traductionsIn=input.readlines()
with open(nomLexique, 'rb') as input:
    lexique=yaml.load(input)

In [4]:
def getBottomList(dictLexique):
    listResult=[]
    for cle in dictLexique:
        if isinstance(dictLexique[cle],list):
            listResult.append(dictLexique[cle])
        else:
            listResult.extend(getBottomList(dictLexique[cle]))
    return listResult

def findFormes(formeIN):
    forme=formeIN.strip()
    for verbe in verbes:
        if forme in verbe:
            vsg=verbe[1::2]
            vpl=verbe[2::2]
            if forme in vsg:
                nombre="sg"
            elif forme in vpl:
                nombre="pl"
            else:
                print "problème avec la personne du verbe"
    return (nombre,vsg,vpl)

In [5]:
verbes=getBottomList(lexique["VER"])
findFormes("tombait")

('sg', ['tombe', 'tombait'], ['tombent', 'tombaient'])

In [6]:
def getAlternatives(chaine):
    result=set()
    resultSG=set()
    resultPL=set()
    alternatives=chaine.split(",")
    for alternative in alternatives:
        if alternative.split()[0] in ["MS","FS","MV","FV",
                                      "le","la","l","ce","cet","cette","un","une",
                                      "Nicole","Nabil","Katisha"]:
            storeResult=resultSG
        elif alternative.split()[0] in ["PL",
                                        "les","ces","des",
                                        "deux","trois","quatre"]:
            storeResult=resultPL
        else:
            storeResult=result
        alternative=alternative.strip()
        if "MS" in alternative:
            for det in ["le","ce","un"]:
                storeResult.add(alternative.replace("MS",det))
        elif "MV" in alternative:
            for det in ["l'","cet","un"]:
                storeResult.add(alternative.replace("MV",det).replace("' ","'"))
        elif "FS" in alternative:
            for det in ["la","cette","une"]:
                storeResult.add(alternative.replace("FS",det))
        elif "FV" in alternative:
            for det in ["l'","cette","une"]:
                storeResult.add(alternative.replace("FV",det).replace("' ","'"))
        elif "PL" in alternative:
            for det in ["les","ces","des","les deux","ces deux","deux"]:
                storeResult.add(alternative.replace("PL",det))
        elif "AUX" in alternative:
            for det in ["aux",u"à ces",u"à des","aux deux",u"à ces deux",u"à deux"]:
                storeResult.add(alternative.replace("AUX",det))
        elif "AU" in alternative:
            for det in ["au",u"à ce",u"à un"]:
                storeResult.add(alternative.replace("AU",det))
        elif "DU" in alternative:
            for det in ["du","de ce","d'un"]:
                storeResult.add(alternative.replace("DU",det))
        elif "DES" in alternative:
            for det in ["des","de ces","de","des deux","de ces deux","de deux"]:
                storeResult.add(alternative.replace("DES",det))
        else:
            storeResult.add(alternative)
    return result,resultSG,resultPL         

In [7]:
findFormes("dormaient ")

('pl', ['dort', 'dormait', 'dormit'], ['dorment', 'dormaient', 'dormirent'])

In [8]:
phrasesOut=set()
verbes=getBottomList(lexique["VER"])
for ligne in traductionsIn[:]:
    syntagmes=ligne.strip().split("\t")
    alternatives={'':[""]}
    gnSG={}
    gnPL={}
    vSG=[]
    vPL=[]
    divers={}
    print " ".join(syntagmes)
    (nombreSujet,vSG,vPL)=findFormes(syntagmes[1])
    if len(vSG)>2:
        vSG=vSG[:2]
    if len(vPL)>2:
        vPL=vPL[:2]
    for nSyntagme,syntagme in enumerate(syntagmes):
        if syntagme:
            if nSyntagme==0:
                gnSG,gnPL=getAlternatives(syntagme)[1:]
            elif nSyntagme==2:
                codSG,codPL=getAlternatives(syntagme)[1:]
                alternatives[nSyntagme]=codSG|codPL
            elif not nSyntagme in [1]:
                alternatives[nSyntagme]=getAlternatives(syntagme)[0]
    altSyntagmes=[list(alternatives[k]) if k in alternatives else [""] for k in range(2,len(syntagmes))]
    if gnSG:
        sujetSG=[gnSG,vSG]+altSyntagmes
        for phrase in it.product(*sujetSG):
            phrase=list(phrase)
            if not phrase[-1].endswith("."): phrase[-1]+="."
            phrasesOut.add("\t".join(phrase))
    if gnPL:
        sujetPL=[gnPL,vPL]+altSyntagmes
        for phrase in it.product(*sujetPL):
            phrase=list(phrase)
            if not phrase[-1].endswith("."): phrase[-1]+="."
            phrasesOut.add("\t".join(phrase))

MS chasseur,PL chasseurs offrent MS coyote,PL coyotes à FV infirmière,AUX infirmières
FV infirmière,PL infirmières offre MS coyote AU chasseur, AUX chasseurs
MS fruit,PL fruits arrivaient   dans FS cuisine de Nicole,dans PL cuisines de Nicole
Nicole arrive   dans FS cuisine des fruits,dans la cuisine DES fruits
SG petite table jaune,PL petites tables jaunes tombent   devant les trois chats de Nabil,devant PL chats de Nabil,devant MS chat de Nabil dans FS cuisine
les trois chats de Nabil,PL chats de Nabil,MS chat de Nabil tombent   devant FS petite table jaune,devant PL petites tables jaunes dans FS cuisine.
SG grand chasseur de coyotes donne FS maison AUX infirmières du village.
PL infirmières du village donnent FS maison AU grand chasseur de coyotes.
les trois villages de la plaine,PL villages de la plaine chassent PL chasseurs d'autruches   avec PL balais.
MS chasseur d'autruches,PL chasseurs d'autruches chassent les trois villages de la plaine  avec MS balai,avec PL balais.
Katisha 

In [9]:
for nPhrase,phrase in enumerate(sorted(phrasesOut)):
    print nPhrase, phrase
    

0 Katisha	achetait	la viande d'un coyote	aux deux petits chasseurs	avec ce poisson.
1 Katisha	achetait	la viande d'un coyote	aux deux petits chasseurs	avec le poisson.
2 Katisha	achetait	la viande d'un coyote	aux deux petits chasseurs	avec un poisson.
3 Katisha	achetait	la viande d'un coyote	aux petits chasseurs	avec ce poisson.
4 Katisha	achetait	la viande d'un coyote	aux petits chasseurs	avec le poisson.
5 Katisha	achetait	la viande d'un coyote	aux petits chasseurs	avec un poisson.
6 Katisha	achetait	la viande d'un coyote	à ces deux petits chasseurs	avec ce poisson.
7 Katisha	achetait	la viande d'un coyote	à ces deux petits chasseurs	avec le poisson.
8 Katisha	achetait	la viande d'un coyote	à ces deux petits chasseurs	avec un poisson.
9 Katisha	achetait	la viande d'un coyote	à ces petits chasseurs	avec ce poisson.
10 Katisha	achetait	la viande d'un coyote	à ces petits chasseurs	avec le poisson.
11 Katisha	achetait	la viande d'un coyote	à ces petits chasseurs	avec un poisson.
12 Katis

665 ce garçon de la maison de Nicole	achetait	les deux autruches		pour deux filles de Nabil.
666 ce garçon de la maison de Nicole	achetait	les deux autruches		pour les deux filles de Nabil.
667 ce garçon de la maison de Nicole	achetait	les deux autruches		pour les filles de Nabil.
668 ce garçon de la maison de Nicole	achète	ces autruches		pour ces deux filles de Nabil.
669 ce garçon de la maison de Nicole	achète	ces autruches		pour ces filles de Nabil.
670 ce garçon de la maison de Nicole	achète	ces autruches		pour des filles de Nabil.
671 ce garçon de la maison de Nicole	achète	ces autruches		pour deux filles de Nabil.
672 ce garçon de la maison de Nicole	achète	ces autruches		pour les deux filles de Nabil.
673 ce garçon de la maison de Nicole	achète	ces autruches		pour les filles de Nabil.
674 ce garçon de la maison de Nicole	achète	ces deux autruches		pour ces deux filles de Nabil.
675 ce garçon de la maison de Nicole	achète	ces deux autruches		pour ces filles de Nabil.
676 ce garço

875 ces chasseurs	offrent	ce coyote	à cette infirmière.
876 ces chasseurs	offrent	ce coyote	à des infirmières.
877 ces chasseurs	offrent	ce coyote	à deux infirmières.
878 ces chasseurs	offrent	ce coyote	à l'infirmière.
879 ces chasseurs	offrent	ce coyote	à une infirmière.
880 ces chasseurs	offrent	ces coyotes	aux deux infirmières.
881 ces chasseurs	offrent	ces coyotes	aux infirmières.
882 ces chasseurs	offrent	ces coyotes	à ces deux infirmières.
883 ces chasseurs	offrent	ces coyotes	à ces infirmières.
884 ces chasseurs	offrent	ces coyotes	à cette infirmière.
885 ces chasseurs	offrent	ces coyotes	à des infirmières.
886 ces chasseurs	offrent	ces coyotes	à deux infirmières.
887 ces chasseurs	offrent	ces coyotes	à l'infirmière.
888 ces chasseurs	offrent	ces coyotes	à une infirmière.
889 ces chasseurs	offrent	ces deux coyotes	aux deux infirmières.
890 ces chasseurs	offrent	ces deux coyotes	aux infirmières.
891 ces chasseurs	offrent	ces deux coyotes	à ces deux infirmières.
892 ces chasseurs	

1941 ces deux œufs sur les coussins	tombent			sous le lit dans la cuisine	devant ce poisson rouge.
1942 ces deux œufs sur les coussins	tombent			sous le lit dans la cuisine	devant le poisson rouge.
1943 ces deux œufs sur les coussins	tombent			sous le lit dans la cuisine	devant un poisson rouge.
1944 ces filles de Nabil	achetaient	ces autruches		pour ces deux garçons de la maison de Nicole.
1945 ces filles de Nabil	achetaient	ces autruches		pour ces garçons de la maison de Nicole.
1946 ces filles de Nabil	achetaient	ces autruches		pour des garçons de la maison de Nicole.
1947 ces filles de Nabil	achetaient	ces autruches		pour deux garçons de la maison de Nicole.
1948 ces filles de Nabil	achetaient	ces autruches		pour les deux garçons de la maison de Nicole.
1949 ces filles de Nabil	achetaient	ces autruches		pour les garçons de la maison de Nicole.
1950 ces filles de Nabil	achetaient	ces deux autruches		pour ces deux garçons de la maison de Nicole.
1951 ces filles de Nabil	achetaient	ce

3387 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec des poissons.
3388 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec deux poissons.
3389 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec le poisson.
3390 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec les deux poissons.
3391 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec les poissons.
3392 des petits chasseurs	achetaient	la viande du coyote	à Katisha	avec un poisson.
3393 des petits chasseurs	achètent	la viande d'un coyote	à Katisha	avec ce poisson.
3394 des petits chasseurs	achètent	la viande d'un coyote	à Katisha	avec ces deux poissons.
3395 des petits chasseurs	achètent	la viande d'un coyote	à Katisha	avec ces poissons.
3396 des petits chasseurs	achètent	la viande d'un coyote	à Katisha	avec des poissons.
3397 des petits chasseurs	achètent	la viande d'un coyote	à Katisha	avec deux poissons.
3398 des petits chasseurs	achètent	la viande d'u

4738 le chasseur	offre	un coyote	aux infirmières.
4739 le chasseur	offre	un coyote	à ces deux infirmières.
4740 le chasseur	offre	un coyote	à ces infirmières.
4741 le chasseur	offre	un coyote	à cette infirmière.
4742 le chasseur	offre	un coyote	à des infirmières.
4743 le chasseur	offre	un coyote	à deux infirmières.
4744 le chasseur	offre	un coyote	à l'infirmière.
4745 le chasseur	offre	un coyote	à une infirmière.
4746 le chasseur d'autruches	chassait	les trois villages de la plaine		avec ce balai.
4747 le chasseur d'autruches	chassait	les trois villages de la plaine		avec ces balais.
4748 le chasseur d'autruches	chassait	les trois villages de la plaine		avec ces deux balais.
4749 le chasseur d'autruches	chassait	les trois villages de la plaine		avec des balais.
4750 le chasseur d'autruches	chassait	les trois villages de la plaine		avec deux balais.
4751 le chasseur d'autruches	chassait	les trois villages de la plaine		avec le balai.
4752 le chasseur d'autruches	chassait	les trois villa

6161 les deux villages de la plaine	chassent	deux chasseurs d'autruches		avec les deux balais.
6162 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec ces balais.
6163 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec ces deux balais.
6164 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec des balais.
6165 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec deux balais.
6166 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec les balais.
6167 les deux villages de la plaine	chassent	les chasseurs d'autruches		avec les deux balais.
6168 les deux villages de la plaine	chassent	les deux chasseurs d'autruches		avec ces balais.
6169 les deux villages de la plaine	chassent	les deux chasseurs d'autruches		avec ces deux balais.
6170 les deux villages de la plaine	chassent	les deux chasseurs d'autruches		avec des balais.
6171 les deux villages de la plaine	chassent	les deux chasseurs d'autruch

In [10]:
with codecs.open(nomTraductions.replace(".csv","-extend.csv"),"w",encoding="utf8") as output:
    for phrase in sorted(phrasesOut):
        output.write(phrase+"\n")

In [11]:
fonctions=["Sujet","Verbe","COD","COI","COMP","AJOUT"]
for traduction in traductionsIn[:]:
    for fonction in fonctions:
        traduction=traduction.replace("\t",":"+fonction+u" | ",1)
    print traduction.strip()

MS chasseur,PL chasseurs:Sujet | offrent:Verbe | MS coyote,PL coyotes:COD | à FV infirmière,AUX infirmières:COI | :COMP |
FV infirmière,PL infirmières:Sujet | offre:Verbe | MS coyote:COD | AU chasseur, AUX chasseurs:COI | :COMP |
MS fruit,PL fruits:Sujet | arrivaient:Verbe | :COD | :COI | dans FS cuisine de Nicole,dans PL cuisines de Nicole:COMP |
Nicole:Sujet | arrive:Verbe | :COD | :COI | dans FS cuisine des fruits,dans la cuisine DES fruits
SG petite table jaune,PL petites tables jaunes:Sujet | tombent:Verbe | :COD | :COI | devant les trois chats de Nabil,devant PL chats de Nabil,devant MS chat de Nabil:COMP | dans FS cuisine
les trois chats de Nabil,PL chats de Nabil,MS chat de Nabil:Sujet | tombent:Verbe | :COD | :COI | devant FS petite table jaune,devant PL petites tables jaunes:COMP | dans FS cuisine.
SG grand chasseur de coyotes:Sujet | donne:Verbe | FS maison:COD | AUX infirmières du village.:COI | :COMP |
PL infirmières du village:Sujet | donnent:Verbe | FS maison:COD | AU gr